# Lecture 1: Exericse

# Introducing a pension scheme to the Code 

* The equations below introduce a balanced-budget pay-as-you-go pension scheme to the model outlined in Lecture 1. Working agents contribute a constant share of their wage income $\tau$ to such a pension scheme, while retired agents receive a benefit $b_t$.

###  The balanced budget
\begin{align}
       \tau w_t \sum_{j=1}^{R} l_{j,t-j} & = \sum_{j=R+1}^{T} b_{t} \\
\end{align}

###  The household problem
With a pension system, the household problem reads:
\begin{align}
\max_{\left\{ c_{t}^{i},s_{t}^{i}\right\} }\text{ }U & =\sum_{j=1}^{T}\beta^{t}u(c_{j,t})\\
& s.t.\\
c_{j,t}+a_{j,t} & =\left( 1-\tau \right) w_{t+j}l_{j,t}+\left(1+r_{t+j}\right)a_{j-1,t} + \left(1-l_{j,t}\right)b_t \quad \quad \text{with }  a_{0,t}=0
\end{align}


### Consolidated budget constraint
Deriving the consolidated budget constraint at birth yields:
\begin{align}
\sum_{j=1}^{T}\dfrac{c_{j,t}}{\prod_{x=1}^{j}\left(1+r_{t+x}\right)}= \sum_{j=1}^{T}\dfrac{\left( 1-\tau \right) w_{t+j}l_{j,t} +\left(1-l_{j,t}\right)b_{t+j}}{\prod_{x=1}^{j}\left(1+r_{t+x}\right)}
\end{align}

### Solve for $c_1$
Solving for $c_{1,t}$ by eliminating $c_{j,t}$ using the long-term Euler-Equation:
\begin{align}
c_{1,t}=\dfrac{\sum_{j=1}^{T}\dfrac{\left( 1-\tau \right) w_{t+j}l_{j,t} +\left(1-l_{j,t}\right)b_{t+j}}{\prod_{x=1}^{j}\left(1+r_{t+x}\right)}}{\sum_{j=1}^{T}\dfrac{\left[\dfrac{\beta^{j-1}}{\left(1+r_{t+j}\right)}\prod_{x=1}^{j}\left(1+r_{t+x}\right)\right]^{\frac{1}{\rho}}}{\prod_{x=1}^{j}\left(1+r_{t+x}\right)}}
\label{eq:c_1_fl} \tag{1}
\end{align}

## Steady State
The closed-form solution for first-period consumption for given steady-state factor prices reads:
\begin{equation}
c_1=\dfrac{\sum_{j=1}^{T}\dfrac{\left( 1-\tau \right) w l_{j}  +   \left(1-l_{j}\right)b }{\left(1+r\right)^{j}}}{\sum_{j=1}^{T}\dfrac{\left[\beta\left(1+r\right)\right]^{\frac{j-1}{\rho}}}{\left(1+r\right)^{j}}}
\end{equation}

where:
\begin{align}
       b = & \tau w \dfrac{  \sum_{j=1}^{R}  l_{j} }{T-R}
\end{align}

steady-state saving can then be traced back via the period-by-period budget.


# Exercise

### 1) Add the pension scheme to the code offered in Lecture 1.

### 2) Solve for the steady-state given $\tau=0.1$ and compare the new steady state to the one without the pension scheme.



# Solution

In [2]:
# Toolkit on coding multiperiod OLG models with pensions
# Iowa State Univeristy, Fall semester 2021
# Authors: Frederik Bjørn Christensen, Tim Dominik Maurer

In [3]:
# Import packages
import numpy as np
import scipy.optimize as optimize
import time
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]
from IPython import display

In [4]:
# Define a dictionary for model parameters
par = dict() 
# Calibrate parameters
par['T'] = 20 # max age
par['R'] = 15 # retirement age
par['α'] = 1/3 # output elasticities of capital
par['ρ'] = 2 # risk aversion parameter
par['δ'] = 0.07 # yearly discount rate
par['β'] = 0.95 # agent's patience parameter
# vector exogenous labour supply given retirement
par['l'] = np.concatenate([np.ones(par['R']),np.zeros([par['T'] - par['R']])]) 
par['L'] = sum(par['l']) # aggregate labour supply

# unpack
T = par['T']
R = par['R']
α = par['α']
ρ = par['ρ']
δ = par['δ']
β = par['β']
l = par['l']
L = par['L']

### Define new functions with a pension system

In [5]:
# Solve for the consumption and savings plan given an interest rate
def solve_paygo(K_guess,par): # Solve for consumption, savings and aggregate capital given a guess for r
    # Unpacking Parameters
    T = par['T']
    R = par['R']
    α = par['α']
    ρ = par['ρ']
    δ = par['δ']
    β = par['β']
    l = par['l']
    L = par['L']
    τ = par['τ']
    
    # STEP 2: Solve for wage given guessed interest rate
    r = α*(K_guess/L)**(α-1) - δ
    w = (1 - α)*(K_guess/L)**α

    # STEP 3: Solve for first periond consumption (we have a closed form in (1))
    b = τ*w*sum(l)/(T-R) # solve for the steady state pension benefit
    C1 = sum(( (1-τ)*l*w + (1-l)*b)/(1+r)**np.linspace(1,T,T))/sum((β*(1+r))**((np.linspace(1,T,T)-1)/ρ)/(1+r)**np.linspace(1,T,T))
    
    # STEP 4: Solve for the whole consumption path using the Euler equation
    C = C1*(β*(1+r))**((np.arange(T))/ρ)

    # STEP 5: Solve for the whole saviongs path using the budget constraint
    A = np.zeros(T) # preallocate storage
    A[0] = (1-τ)*w*l[0] - C[0] # solve for first period savings given no initial wealth
    for t in range(T): # solve the whole savings path
        if t>0:
            A[t] = (1-τ)*w*l[t] + (1-l[t])*b + (1 + r)*A[t-1] - C[t]

    # STEP 5: Compute implied aggregate capital by summing over savings path
    K = sum(A)
     
    return C,A,K


# new objective function calling solve_paygo
def objective_paygo(K_guess,par): # Solves the SS
    # Calling the solve_paygo to determine Consumption, Savings, and the implies Capital from the current guess
    C,A,K_implied = solve_paygo(K_guess,par) 
    
    # STEP 8: Check distance between K_guess - K_implied (Note we define the loss as the squared difference)
    loss = (K_guess - K_implied)**2  
    return loss

### Solve for new Steady State and compare it to the old without a pension scheme

In [6]:
# Define a model-consistent level of the lower bound
r_lb = 1e-4 - δ # lower bound r
K_ub = L*((r_lb+δ)/α)**(1/(α-1)) # upper bound K(r_lb)

# Add a zero pension contribution rate
par['τ'] = 0.0
sol_ss = optimize.minimize_scalar(objective_paygo,bounds=[0,K_ub],args=(par),method='bounded',options={'xatol': 1e-4, 'maxiter': 10000})
K_ss = sol_ss.x # Equilibrium capital stock

# STEP 9: Minimize loss
# Add a non-zero pension contribution rate
par['τ'] = 0.1
sol_paygo = optimize.minimize_scalar(objective_paygo,bounds=[0,K_ub],args=(par),method='bounded',options={'xatol': 1e-4, 'maxiter': 10000})
K_ss_paygo = sol_paygo.x # Equilibrium capital stock

print('Steady-state capital without pension scheme:',K_ss)
print('Steady-state capital with pension scheme:   ',K_ss_paygo)

Steady-state capital without pension scheme: 38.7366233754656
Steady-state capital with pension scheme:    30.610747375607385


### Intution
Compared to a setting without pensions, agents do not need to save as much during work-life in order to maintain the same level of concumption because they will receive a pay-as-you-go pension benefit in retirement. Hence, savings fall and there is less capital in steady-state.

## Going Forward
In the next lecture, we will consider the model with a pension system in more detail. Specifically, we will go over how to solve the dynamic transition path between two steady states when policy changes unexpectedly (an MIT-shock).